# Model the Data
Given a set of scalar data, you can model it with a suitable pdf. This exercise will help you learn the following:
- How would you choose the best model to fit the given data?
- How would you estimate the model parameters from the given data?
- Given a model, how do you sample new data from it?

Note: You are allowed to use only numpy and matplotlib libraries. No ML library.

In [352]:
import numpy as np
np.random.seed(111)

In [353]:
def data2gaussian(S):
    '''
    Return optimal parameters - (mu,sigma)
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        mu: float
        sigma: float
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    S = np.array(S)
    mu = np.sum(S)/S.size
    S = (S-mu)**2
    sigma = np.sqrt(np.sum(S)/S.size)
    return mu, sigma

In [354]:
def test_data2gaussian(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    mu, sigma = data2gaussian(S)
    print(mu,sigma)
    assert isinstance(mu, (int, float))
    assert isinstance(sigma, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2gaussian()

0.06000000000000001 0.19595917942265426
Test passed 👍


In [355]:
def data2laplacian(S):
    '''
    Return optimal parameters - (mu,b). See https://en.wikipedia.org/wiki/Laplace_distribution
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        mu: float
        b: float
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    S = np.array(S)
    mu = np.median(S)
    S = abs(S-mu)
    b = np.sum(S)/S.size
    return mu, b

In [356]:
def test_data2laplacian(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    mu, b = data2laplacian(S)
    print(mu,b)
    assert isinstance(mu, (int, float))
    assert isinstance(b, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2laplacian()

0.0 0.14
Test passed 👍


In [357]:
def data2uniform(S):
    '''
    Return optimal parameters - (a,b)
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        a: float
        b: float
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    S = np.array(S)
    a = min(S)
    b = max(S)
    return a, b

In [358]:
def test_data2uniform(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    a, b = data2uniform(S)
    print(a,b)
    assert isinstance(a, (int, float))
    assert isinstance(b, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2uniform()

-0.2 0.4
Test passed 👍


In [359]:
def data2model(S):
    '''
    Inputs:
        S: np array of shape (Ns,). These are scalar samples of a random variable.
    Outputs:
        modelName: return one out of these - 'gaussian', 'laplacian', or 
                   'uniform' which best models the data
    '''

    ### WRITE YOUR CODE HERE - 10 MARKS
    S = np.array(S)
    mu_g, sigma_g = data2gaussian(S)
    mu_l, b_l = data2laplacian(S)
    a, b = data2laplacian(S)
    
    Sg = np.copy(S)
    Sl = np.copy(S)
    Su = np.copy(S)
    
    Sg = np.log(1/(np.sqrt(2*np.pi)*sigma_g)) - ( (0.5/(sigma_g**2)) * ((Sg - mu_g)**2) )
    Sl = np.log(1/(2*b_l)) - ( (1/b) * abs(Sl - mu_l) )
    Su = np.log(1/(b-a)) + np.zeros(Su.shape)
    
    lik_g = np.sum(Sg)
    lik_l = np.sum(Sl)
    lik_u = np.sum(Su)
    
    if lik_g > lik_l and lik_g > lik_u:
        modelName = 'gaussian'
    elif lik_l > lik_g and lik_l > lik_u:
        modelName = 'laplacian'
    else:
        modelName = 'uniform'
    
    return modelName

In [360]:
def test_data2model(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    modelName = data2model(S)
    assert modelName in ['gaussian', 'laplacian', 'uniform']
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2model()

Test passed 👍


In [361]:
def sampleGMM(pi, mu, sigma, Ns=1):
    '''
    Inputs:
        pi: np.array of shape (K,), p(z_k)
        mu: np.array of shape (K,), mu of kth gaussian
        sigma: np.array of shape (K,), sigma of kth gaussian
        Ns: int, number of samples
    Outputs:
        S: np.array of shape (Ns,), samples from the GMM
    '''

    ### WRITE YOUR CODE HERE - 10 MARKS
    pi = np.array(pi)
    mu = np.array(mu)
    sigma = np.array(sigma)
    S = np.array([])
    
    for s in range(Ns):
        k = np.random.choice(pi.shape[0],1,p=pi)
        S = np.append(S,np.random.normal(mu[k],sigma[k],1))
    
    return S

In [362]:
def test_sampleGMM(): # checks formatting only
    pi = [0.3,0.7]
    mu = [-1.1, 1.3]
    sigma = [1.5, 0.4]
    Ns = 5
    S = sampleGMM(pi, mu, sigma, Ns)
    assert S.shape==(5,)
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_sampleGMM()

Test passed 👍


In [363]:
def data2GMM(S, K):
    '''
    Return optimal parameters - (pi,mu,sigma)
    Inputs:
        S: np array of shape (Ns,Na). These are samples of a random variable. Na can be 1, 2 or 3
    Outputs:
        pi: np array of shape (K,)
        mu: np array of shape (K,Na)
        sigma: np array of shape (K,Na,Na)
    '''

    ### WRITE YOUR CODE HERE - 15 MARKS
    e = 0.00001  #Introduce bias in the system for its stability
    # Initialization of parameters
    S = np.array(S)
    pi = np.array([1/K]*K)
    mu = np.random.uniform(-5,5,(K,S.shape[1]))
    var = np.array([np.identity(S.shape[1]) + (e*np.identity(S.shape[1]))]*K)
    r = np.zeros((S.shape[0],K))
    max_iter = 100
    #Eo = 0
    # Allowed maximum iterations, if the convergence is achieved then it will jumble around the convergent point
    for n in range(max_iter):
        
        for i in range(S.shape[0]):
            for k in range(K):
                x = (S[i]-mu[k]).reshape((1,S.shape[1]))
                r[i,k] = pi[k]*(abs(np.linalg.det(var[k]))**-0.5)*np.exp(-0.5*np.linalg.det(x @ np.linalg.inv(var[k]) @ x.T))

        r = (r.T/r.sum(axis=1)).T
        
        for k in range(K):
            pi[k] = r[:,k].sum()/S.shape[0]

            s1 = 0
            for i in range(S.shape[0]):
                s1 += r[i,k]*S[i]
            mu[k] = s1/r[:,k].sum()

            s2 = 0
            for i in range(S.shape[0]):
                x = (S[i]-mu[k]).reshape((1,S.shape[1]))
                s2 += r[i,k]*(x.T @ x)
            # Update covariance matrix with the bias of e in its digonal elements
            var[k] = (s2/r[:,k].sum()) + (e*np.identity(S.shape[1]))
        '''
        E = 0
        for i in range(S.shape[0]):
            for k in range(K):
                x = (S[i]-mu[k]).reshape((1,S.shape[1]))
                E += r[i,k]*np.log(pi[k]*((2*np.pi*abs(np.linalg.det(var[k])))**-0.5)*np.exp(-0.5*np.linalg.det(x @ np.linalg.inv(var[k]) @ x.T)))
                print('n',n,'i',i,'k',k,'E',E,'r',r[i,k],np.exp(-0.5*np.linalg.det(x @ np.linalg.inv(var[k]) @ x.T)),-0.5*np.linalg.det(x @ np.linalg.inv(var[k]) @ x.T))
        if n!=0 and abs((E-Eo)/Eo)<0.01:
            break
        Eo = E
        '''
    for k in range(K):
        var[k] = np.sqrt(np.diag(np.diag(var[k])))
    sigma = var
    
    return pi, mu, sigma


In [364]:
def test_data2GMM(): # checks formatting only
    S = np.random.random((10,3))
    pi, mu, sigma = data2GMM(S,2)
    assert pi.shape==(2,)
    assert mu.shape==(2,3)
    assert sigma.shape==(2,3,3)
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2GMM()

Test passed 👍
